<a href="https://colab.research.google.com/github/royal-dit/ML-algorithms/blob/main/Logistic%20regression/Stock_price_prediciton_using_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance


In [ ]:
import pandas as pd
import yfinance as yf
import statsmodels.api as sm

In [ ]:
data = yf.download("^GSPC",start = '2001-01-03',end ='2022-11-01')

[*********************100%***********************]  1 of 1 completed


In [ ]:
data.head(5)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2001-01-03,1283.270020,1347.760010,1274.619995,1347.560059,1347.560059,1880700000
2001-01-04,1347.560059,1350.239990,1329.140015,1333.339966,1333.339966,2131000000
2001-01-05,1333.339966,1334.770020,1294.949951,1298.349976,1298.349976,1430800000
2001-01-08,1298.349976,1298.349976,1276.290039,1295.859985,1295.859985,1115500000
2001-01-09,1295.859985,1311.719971,1295.140015,1300.800049,1300.800049,1191300000


In [ ]:
df = data["Adj Close"].pct_change()*100

In [ ]:
df = df.rename("Today")


In [ ]:
df = df.reset_index()

,Date,Today
0,2001-01-03,NaN
1,2001-01-04,-1.055247
2,2001-01-05,-2.624236
3,2001-01-08,-0.191781
4,2001-01-09,0.381219
...,...,...
5487,2022-10-25,1.626665
5488,2022-10-26,-0.738772
5489,2022-10-27,-0.608261
5490,2022-10-28,2.462638


In [ ]:
for i in range(1,6):
  df["Lag"+str(i)] = df['Today'].shift(i)

In [ ]:
df["Volume"] = data.Volume.shift(1).values/100_000_000

In [ ]:
df = df.dropna()

In [ ]:
df.head(2)

,Date,Today,Lag1,Lag2,Lag3,Lag4,Lag5,Volume
6,2001-01-11,1.031770,0.958639,0.381219,-0.191781,-2.624236,-1.055247,12.965
7,2001-01-12,-0.623287,1.031770,0.958639,0.381219,-0.191781,-2.624236,14.112


In [ ]:
df["Direction"] = [1 if i>0 else 0 for i in df["Today"]]

In [ ]:
df.head(5)

,Date,Today,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Direction,const
6,2001-01-11,1.031770,0.958639,0.381219,-0.191781,-2.624236,-1.055247,12.965,1,1
7,2001-01-12,-0.623287,1.031770,0.958639,0.381219,-0.191781,-2.624236,14.112,0,1
8,2001-01-16,0.614309,-0.623287,1.031770,0.958639,0.381219,-0.191781,12.760,1,1
9,2001-01-17,0.212561,0.614309,-0.623287,1.031770,0.958639,0.381219,12.057,1,1
10,2001-01-18,1.391532,0.212561,0.614309,-0.623287,1.031770,0.958639,13.491,1,1


In [ ]:
df =sm.add_constant(df)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
df["const"] = 1

In [ ]:
df

,Date,Today,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Direction,const
6,2001-01-11,1.031770,0.958639,0.381219,-0.191781,-2.624236,-1.055247,12.9650,1,1
7,2001-01-12,-0.623287,1.031770,0.958639,0.381219,-0.191781,-2.624236,14.1120,0,1
8,2001-01-16,0.614309,-0.623287,1.031770,0.958639,0.381219,-0.191781,12.7600,1,1
9,2001-01-17,0.212561,0.614309,-0.623287,1.031770,0.958639,0.381219,12.0570,1,1
10,2001-01-18,1.391532,0.212561,0.614309,-0.623287,1.031770,0.958639,13.4910,1,1
...,...,...,...,...,...,...,...,...,...,...
5487,2022-10-25,1.626665,1.188198,2.372482,-0.795091,-0.667210,1.142757,47.4793,1,1
5488,2022-10-26,-0.738772,1.626665,1.188198,2.372482,-0.795091,-0.667210,48.4312,0,1
5489,2022-10-27,-0.608261,-0.738772,1.626665,1.188198,2.372482,-0.795091,48.1731,0,1
5490,2022-10-28,2.462638,-0.608261,-0.738772,1.626665,1.188198,2.372482,46.8732,1,1


In [ ]:
split_size =0.8

In [ ]:
X = df[["const","Lag1","Lag2","Lag3","Lag4","Lag5","Volume"]]


In [ ]:
y = df.Direction 

In [ ]:
#make train/test splits
def make_train_test_splits(windows,labels,test_split=0.2):
   split_size = int(len(windows)*(1-test_split)) #this seperate data in 80 20 
   train_windows = windows[:split_size]
   train_labels = labels[:split_size] 
   test_windows = windows[split_size:]
   test_labels = labels[split_size:]

   return train_windows,test_windows,train_labels,test_labels

In [ ]:
X_train,X_test,y_train,y_test = make_train_test_splits(X,y)
len(X_train)

4388

#using Logistic reg

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
log_reg = LogisticRegression()

In [ ]:
log_reg.fit(X_train,y_train)


LogisticRegression()

In [ ]:
y_pred  = log_reg.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
accuracy_score(y_test,y_pred)


0.5537340619307832

#using Stats model.api

In [ ]:
model = sm.Logit(y,X)


In [ ]:
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.688263
         Iterations 4


In [ ]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Direction   No. Observations:                 5486
Model:                          Logit   Df Residuals:                     5479
Method:                           MLE   Df Model:                            6
Date:                Sat, 05 Nov 2022   Pseudo R-squ.:                0.002986
Time:                        16:09:37   Log-Likelihood:                -3775.8
converged:                       True   LL-Null:                       -3787.1
Covariance Type:            nonrobust   LLR p-value:                 0.0009354
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1017      0.069      1.481      0.139      -0.033       0.236
Lag1          -0.0975      0.023     -4.322      0.000      -0.142      -0.053
Lag2           0.0075      0.022      0.334      0.739      -0.036       0.051
Lag3           0.0100      0.022      0.446      0.656      -0.034       0.054
Lag4          -0.0113      0.022     -0.505      0.613      -0.055       0.033
Lag5          -0.0309      0.022     -1.386      0.166      -0.075       0.013
Volume         0.0016      0.002      0.840      0.401      -0.002       0.005
==============================================================================
"""

In [ ]:
prediction  = result.predict(X)
prediction[:3]


6    0.522957
7    0.529374
8    0.550228
dtype: float64

In [ ]:
def con_matrix(act,pred):
  predtrans = ["Up" if i>0.5 else "Down" for i in pred]
  actuals = ["Up" if i>0 else "Down" for i in act]
  confusion_matrix = pd.crosstab(pd.Series(actuals),
                                 pd.Series(predtrans),
                                 rownames = ["Actual"],
                                 colnames = ["Predicted"])
  return confusion_matrix
  
                                 
                                 
  

In [ ]:
con_matrix(y,prediction)

Predicted,Down,Up
Actual,,
Down,214,2323
Up,183,2766


In [ ]:
len(df)

5486

In [ ]:
(214+2766)/5486

0.5432008749544295

In [ ]:
# from sklearn.metrics import accuracy_score
# acc = accuracy_score(y,prediction)